In [2]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy as copy
import datetime
import json
from math import ceil
import multiprocessing
import logging
import operator
import os
from pathlib import Path
import random
import sys
import time
import typing
import warnings

import joblib
from joblib import delayed, Parallel
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from numpy.core.numeric import outer
import pandas as pd
from scipy.stats import mode, entropy
import seaborn as sns
import sklearn.ensemble
import sklearn.feature_selection
import sklearn.metrics
from sklearn.metrics import roc_auc_score
import sklearn.model_selection
import typer
from tqdm.auto import tqdm

root = str(Path.cwd().parents[1])
sys.path.append(root)
os.chdir(root)
import src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Scenario A

In [2]:
%%time

X, y, s = src.get_Xys()
X.drop(columns='port_stay', inplace=True)
cv = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
clf = sklearn.model_selection.GridSearchCV(
    estimator=sklearn.ensemble.RandomForestClassifier(random_state=42),
    param_grid={'max_depth': np.arange(1, 11)},
    scoring='roc_auc',
    refit=True,
    verbose=2,
    cv=cv,
    n_jobs=50
)
nested_score = sklearn.model_selection.cross_validate(clf, X=X, y=y>1, cv=cv, n_jobs=5, return_train_score=True)

CPU times: user 854 ms, sys: 1.36 s, total: 2.22 s
Wall time: 1min 16s


In [3]:
pd.DataFrame(nested_score).agg({'test_score': ['mean', 'std'], 'train_score': ['mean', 'std']})

,test_score,train_score
mean,0.670713,0.825244
std,0.018844,0.035342


# Scenario B

In [4]:
%%time

X, y, s = src.get_Xys()
X.drop(columns='port_stay', inplace=True)
cv = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)
clf = sklearn.model_selection.GridSearchCV(
    estimator=sklearn.ensemble.RandomForestClassifier(random_state=42),
    param_grid={'max_depth': np.arange(1, 11)},
    scoring='roc_auc',
    refit=True,
    verbose=2,
    cv=cv,
    n_jobs=50
)
nested_score = sklearn.model_selection.cross_validate(clf, X=X, y=y>0, cv=cv, n_jobs=5, return_train_score=True)

CPU times: user 262 ms, sys: 240 ms, total: 502 ms
Wall time: 1min 23s


In [5]:
pd.DataFrame(nested_score).agg({'test_score': ['mean', 'std'], 'train_score': ['mean', 'std']})

,test_score,train_score
mean,0.814629,0.892599
std,0.005918,0.002530
